In [1]:
import os
import sys
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
os.chdir("/mnt/c/Users/Dave/Project/COBS")
from cobs import Model, Reward
from cobs import OccupancyGenerator as OG
from tqdm import tqdm
from pprint import pprint

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from setting import eplus_naming_dict, eplus_var_types

In [2]:
# Set E+ and IDF files paths 
Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")
idf_files_path = "/mnt/c/users/Dave/Downloads/idf-sample-files/"

# A dictionary formatted as {target sensor: state name}
# The target sensor is a tuple: (variable name, key value)

# Initialize the model with idf and weather files
r = Reward()
model = Model(idf_file_name=idf_files_path+"2020/RefBldgLargeOfficeNew2004_Chicago.idf",
# model = Model(idf_file_name="cobs/data/buildings/5ZoneAirCooled.idf",              
              weather_file="cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw",
              eplus_naming_dict=eplus_naming_dict,
              eplus_var_types=eplus_var_types,
              reward = r)

# Let's generate the occupancy schedule with default parameters
# Note that the IDF file may already contain an occupancy schedule
# We can check the schedule of each zone (if any)
# for zone_schedule in model.get_configuration("People"):
#     print(f"Zone {zone_schedule['Zone_or_ZoneList_Name']}"
#           f" is using schedule {zone_schedule['Number_of_People_Schedule_Name']}")

In [3]:
# Set the length of simulation to 1 day
# model.set_runperiod(1, start_year=2020, specify_year=True)
# Set the number of timesteps per hour to 60, so we have minutely data
model.set_timestep(1)

# Run the experiment
state = model.reset()

print(state)
# with tqdm(total=model.get_total_timestep() - 1) as pbar:
#     while not model.is_terminate():
#         counter_before = model.counter
#         print("Before: ", counter_before, model.counter, model.total_timestep, model.is_terminate())
#         state = model.step(None)
#         if model.counter == counter_before:
#             print("=")
#             model.counter += 1
#         print('After: ', counter_before, model.counter, model.total_timestep, model.is_terminate())
#         pbar.update(1)
#         pbar.update(1)


# total=model.get_total_timestep() - 1
# ts = 0
# with tqdm(total=model.get_total_timestep() - 1) as pbar:
#     while ts < total - 1:
#         state = model.step(None)
#         pbar.update(1)
#         ts += 1

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.02.25 23:17
Initializing Response Factors
Calculating CTFs for "MASS NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "UNDERGROUND WALL NON-RES", Construction # 7
Calculating CTFs for "DROPCEILING", Construction # 9
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 10
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables

In [5]:
state['reward']

2.529697436038336

In [6]:
model.step()

{'timestep': 2,
 'time': datetime.datetime(2022, 1, 21, 3, 0),
 'temperature': {'Basement': 19.853137603721898,
  'Core_bottom': 19.378936136378826,
  'Core_mid': 19.122971041905377,
  'Core_top': 17.866802240136003,
  'GroundFloor_Plenum': 16.84818386054854,
  'MidFloor_Plenum': 16.7684796985919,
  'Perimeter_bot_ZN_1': 15.09209177393999,
  'Perimeter_bot_ZN_2': 15.256693968481327,
  'Perimeter_bot_ZN_3': 14.993097337412538,
  'Perimeter_bot_ZN_4': 15.261426451989756,
  'Perimeter_mid_ZN_1': 14.456164857618296,
  'Perimeter_mid_ZN_2': 14.853238617601695,
  'Perimeter_mid_ZN_3': 14.236653273036966,
  'Perimeter_mid_ZN_4': 14.806559632367167,
  'Perimeter_top_ZN_1': 13.524904730164653,
  'Perimeter_top_ZN_2': 14.079555537302847,
  'Perimeter_top_ZN_3': 13.115697367006858,
  'Perimeter_top_ZN_4': 13.998338786267913,
  'TopFloor_Plenum': 5.884624081943815},
 'occupancy': {'Basement': 0.0,
  'Core_bottom': 0.0,
  'Core_mid': 0.0,
  'Core_top': 0.0,
  'Perimeter_bot_ZN_1': 0.0,
  'Perimeter

In [8]:
list(state['PMV'].keys())

['Basement',
 'Core_bottom',
 'Core_mid',
 'Core_top',
 'Perimeter_bot_ZN_1',
 'Perimeter_bot_ZN_2',
 'Perimeter_bot_ZN_3',
 'Perimeter_bot_ZN_4',
 'Perimeter_mid_ZN_1',
 'Perimeter_mid_ZN_2',
 'Perimeter_mid_ZN_3',
 'Perimeter_mid_ZN_4',
 'Perimeter_top_ZN_1',
 'Perimeter_top_ZN_2',
 'Perimeter_top_ZN_3',
 'Perimeter_top_ZN_4']

In [7]:
sys.path.insert(0, '/usr/local/EnergyPlus-9-4-0/')
from pyenergyplus.api import EnergyPlusAPI
import pyenergyplus
pyenergyplus.api.EnergyPlusAPI.api_version()

'0.2'

In [41]:
api = EnergyPlusAPI()
state = api.state_manager.new_state()
x = []
def callback_function(state_argument):
    oa_temp_handle = api.exchange.get_variable_handle(state_argument, u"FACILITY TOTAL HVAC ELECTRIC DEMAND POWER", u"ENVIRONMENT")
    oa_temp = api.exchange.get_variable_value(state_argument, oa_temp_handle)
    x.append(oa_temp)
api.runtime.callback_begin_zone_timestep_after_init_heat_balance(state, callback_function)
api.runtime.run_energyplus(state,
    [
        '-w', 'cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw',
        '-d', 'out',
        "/mnt/c/users/Dave/Downloads/idf-sample-files/2020/RefBldgLargeOfficeNew2004_Chicago.idf"
    ]
)
# api.exchange.get_meter_handle("Heating:EnergyTransfer")

EnergyPlus Starting
EnergyPlus, Version 9.4.0-998c4b761e, YMD=2022.02.21 00:52
Initializing Response Factors
Calculating CTFs for "MASS NON-RES EXT WALL"
Calculating CTFs for "IEAD NON-RES ROOF"
Calculating CTFs for "EXT-SLAB"
Calculating CTFs for "INT-WALLS"
Calculating CTFs for "INT-FLOOR-TOPSIDE"
Calculating CTFs for "UNDERGROUND WALL NON-RES"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INTERIORFURNISHINGS"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar A

EnergyPlus Completed Successfully.


0

In [23]:
len(x)

8070

In [54]:
model.replay.sample(1)

(array([{'time': datetime.datetime(2022, 1, 21, 0, 1), 'occupancy': {'Basement': 0.0, 'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_top': 0.0, 'Perimeter_bot_ZN_1': 0.0, 'Perimeter_bot_ZN_2': 0.0, 'Perimeter_bot_ZN_3': 0.0, 'Perimeter_bot_ZN_4': 0.0, 'Perimeter_mid_ZN_1': 0.0, 'Perimeter_mid_ZN_2': 0.0, 'Perimeter_mid_ZN_3': 0.0, 'Perimeter_mid_ZN_4': 0.0, 'Perimeter_top_ZN_1': 0.0, 'Perimeter_top_ZN_2': 0.0, 'Perimeter_top_ZN_3': 0.0, 'Perimeter_top_ZN_4': 0.0}, 'timestep': 0, 'energy': 0.0, 'temperature': {}, 'terminate': False}],
       dtype=object),
 array([{'actuator': {}, 'global': {}}], dtype=object),
 array([{'time': datetime.datetime(2022, 1, 21, 0, 2), 'occupancy': {'Basement': 0.0, 'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_top': 0.0, 'Perimeter_bot_ZN_1': 0.0, 'Perimeter_bot_ZN_2': 0.0, 'Perimeter_bot_ZN_3': 0.0, 'Perimeter_bot_ZN_4': 0.0, 'Perimeter_mid_ZN_1': 0.0, 'Perimeter_mid_ZN_2': 0.0, 'Perimeter_mid_ZN_3': 0.0, 'Perimeter_mid_ZN_4': 0.0, 'Perimeter_top_ZN_1': 0.0, 'Pe

In [28]:
state

{'timestep': 19,
 'time': datetime.datetime(2022, 1, 21, 20, 0),
 'temperature': {},
 'occupancy': {'Basement': 0.0,
  'Core_bottom': 0.0,
  'Core_mid': 0.0,
  'Core_top': 0.0,
  'Perimeter_bot_ZN_1': 0.0,
  'Perimeter_bot_ZN_2': 0.0,
  'Perimeter_bot_ZN_3': 0.0,
  'Perimeter_bot_ZN_4': 0.0,
  'Perimeter_mid_ZN_1': 0.0,
  'Perimeter_mid_ZN_2': 0.0,
  'Perimeter_mid_ZN_3': 0.0,
  'Perimeter_mid_ZN_4': 0.0,
  'Perimeter_top_ZN_1': 0.0,
  'Perimeter_top_ZN_2': 0.0,
  'Perimeter_top_ZN_3': 0.0,
  'Perimeter_top_ZN_4': 0.0},
 'terminate': False,
 'energy': 3860214094.766755,
 'Site Diffuse Solar Radiation Rate per Area_Environment': 0.0,
 'Ambient Temperature': -20.6}

In [76]:
model.counter

0

In [83]:
model.run_parameters

['-w',
 'cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw',
 '-d',
 'result',
 'input.idf']

In [10]:
def weights_init_(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight, gain=1)
        torch.nn.init.constant_(m.bias, 0)
        
class DQN(nn.Module):
    def __init__(self, num_features, num_actions):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(num_features, 24),
            nn.ReLU(),
            nn.Linear(24, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, num_actions)
        )
        
    def forward(self, x):
        x = self.model(x.float())
        return x

In [1]:
class EplisonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

In [103]:
def norm(X):
    return (X - min(X)) / (max(X) - min(X))
class Agent:
    def __init__(self, strategy):
        # Initialize the Q values randomly
        self.q = np.random.random((16 * 8, 16))
        self.last_action = None
        self.last_state = None
        self.strategy = strategy
        self.alpha = 0.5
        self.gamma = 1
        self.lambda = 100
        self.temp_min = 19
        self.temp_max = 24
        
        
    def reward(self):
        temp_penalty = 0
        
        for _, temp in state["temperature"].items:
            temp_penalty += max(0, temp - self.temp_max) + max(0, self.temp_min - temp)
        temp_penalty *= self.lambda
        
        return -state['energy'] - temp_penalty
        
    def select_action(self, state, policy_net):
        eplison = strategy.get_exploration_rate()
        if eplison > random.random():
            action = random.randrange(self.num_actions)
    def take_action(self, state):
        
        actions = list()
        # Ignore non-business hour
        if state["time"].hour < 8 or state["time"].hour >= 18:
            return actions
        
        # Take actions by epsilon-greedy method
        setpoint = 0
        self.last_state = int(np.round((sum(state["temperature"].values()) - state["temperature"]["PLENUM-1"]) / 5) - 15 + \
                              (state["time"].hour - 9) * 16)
        if np.random.random() < epsilon:
            setpoint = np.random.choice(range(15, 31))
        else:
            setpoint = np.argmax(self.q[self.last_state, :]) + 15
        
        
        # Encode the action
        for zone, temp in state["temperature"].items():
            # The PLENUM-1 has no heating/cooling system, so ignore it
            if zone == "PLENUM-1":
                continue
            actions.append({"value": setpoint,
                            "start_time": state["timestep"],
                            "priority": 0,
                            "component_type": "Zone Temperature Control",
                            "control_type": "Heating Setpoint",
                            "actuator_key": zone})
            actions.append({"value": 40,
                            "start_time": state["timestep"],
                            "priority": 0,
                            "component_type": "Zone Temperature Control",
                            "control_type": "Cooling Setpoint",
                            "actuator_key": zone})
        return actions

    def start(self, state):
        encoded_a_ = self.take_action(state)
        self.last_action = encoded_a_[0]["value"] if encoded_a_ else None
        return encoded_a_
        
    def step(self, state):
        a = self.last_action
        s = self.last_state
        encoded_a_ = self.take_action(state)
        a_ = encoded_a_[0]["value"] if encoded_a_ else None
        s_ = self.last_state

        if a is not None and a_ is not None:
            # Update Q
            self.q[s, a - 15] += self.alpha * (-state["energy"] + self.gamma * self.q[s_, a_ - 15] - self.q[s, a - 15])

        self.last_action = a_
        return encoded_a_

if __name__ == '__main__':
    model = Model(idf_file_name="cobs/data/buildings/5ZoneAirCooled.idf",
                  weather_file="cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw")
    agent = SarsaAgent()
    
    model.set_runperiod(31)
    model.set_timestep(1)
    
    num_episode = 100
    g = np.zeros(num_episode)

    with tqdm(total=model.get_total_timestep() - 1) as pbar:
        for i in tqdm(range(num_episode)):
            pbar.reset()
            state = model.reset()
            action = agent.start(state)
            if 9 <= state["time"].hour < 17:
                g[i] += state["energy"]

            while not model.is_terminate():
                state = model.step(action)
                action = agent.step(state)
                if 9 <= state["time"].hour < 17:
                    g[i] += state["energy"]
                pbar.update(1)
    
    # Expect to see a decrease of total energy use
    plt.plot(g)
    plt.show()

  0%|                                                                                                                                                                                                   | 0/743 [00:00<?, ?it/s]

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.02.19 14:26
Initializing Response Factors
Calculating CTFs for "ROOF-1", Construction # 1
Calculating CTFs for "WALL-1", Construction # 2
Calculating CTFs for "FLOOR-SLAB-1", Construction # 4
Calculating CTFs for "INT-WALL-1", Construction # 5
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Computing Interior Diffuse Solar Exchange through Inte

  0%|▎                                                                                                                                                                                          | 1/743 [00:00<08:24,  1.47it/s]

Warming up
Warming up
Performing Zone Sizing Simulation for Load Component Report
...for Sizing Period: #1 CHICAGO_IL_USA ANNUAL HEATING 99% DESIGN CONDITIONS DB
Warming up
Warming up
Warming up
Warming up
Warming up
Warming up
Performing Zone Sizing Simulation for Load Component Report
...for Sizing Period: #2 CHICAGO_IL_USA ANNUAL COOLING 1% DESIGN CONDITIONS DB/MCWB
Re-zeroing zone sizing arrays
Warming up
Warming up
Warming up
Warming up
Warming up
Warming up
Performing Zone Sizing Simulation
...for Sizing Period: #1 CHICAGO_IL_USA ANNUAL HEATING 99% DESIGN CONDITIONS DB
Warming up
Warming up
Warming up
Warming up
Warming up
Warming up
Performing Zone Sizing Simulation
...for Sizing Period: #2 CHICAGO_IL_USA ANNUAL COOLING 1% DESIGN CONDITIONS DB/MCWB
Calculating System sizing
...for Sizing Period: #1 CHICAGO_IL_USA ANNUAL HEATING 99% DESIGN CONDITIONS DB
Calculating System sizing
...for Sizing Period: #2 CHICAGO_IL_USA ANNUAL COOLING 1% DESIGN CONDITIONS DB/MCWB
Adjusting Air Syst

Program terminated: EnergyPlus Terminated--Error(s) Detected.
  1%|██▏                                                                                                                                                                                     | 9/743 [08:21<11:22:04, 55.76s/it]


KeyboardInterrupt: 

In [ ]:
batch_size = 256
gamma = 0.999
eps_start = 1
eps_end = 0.01
eps_decay = 0.001
target_update = 10
memory_size = 100000
lr = 0.001
num_episodes = 100

device = torch.device("cuda" if torch.cuda.is_available() elese "cpu")
strategy = EplisonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, #actions, device)

policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.stat_dict())
target_net.eval()
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)

for episode in range(num_episodes):
    state = model.reset()